In [12]:
# Import Section

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import math
import csv
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate, Dropout, BatchNormalization, MaxPooling2D

In [13]:
# Import Section

import geopandas as gpd
import numpy as np
from PIL import Image
import os

# Path to the folder with all data
PREFIX = "/Users/moctader/Thesis_code"

# Zoom level
ZOOM_LEVEL = 10


csv_path = f"{PREFIX}/GTK_ASsoil_obs.csv"
base_directory = f"{PREFIX}/output20/"


def read_geo_data(csv_path):
    points = gpd.read_file(csv_path)
    return points
points=read_geo_data(csv_path)


# Data points
points.POINT_X = points.POINT_X.astype("float")
points.POINT_Y = points.POINT_Y.astype("float")

#samples

samples = gpd.GeoDataFrame(
    points.CLASS, crs="EPSG:3067", geometry=gpd.points_from_xy(points.POINT_X, points.POINT_Y)
).to_crs("WGS84")

tile_list = [(point.x, point.y) for point in samples['geometry']]


# Creating image filename

samples["i"] = samples.index
samples["filenames"] = samples.apply(lambda row: f"{row['CLASS']}/image_{row['i']}", axis=1)


# Extracting Latitude and Longitude from GeoDataFrame

def get_lat_from_row(p):
    lon, lat = p.geometry.x, p.geometry.y
    return lat

def get_lon_from_one_column(geometry):
    lon, lat = geometry.x, geometry.y
    return lon

# Calculating Latitude for Each Sample

samples["lat"] = samples.apply(lambda row: get_lat_from_row(row), axis=1)

# Calculating Longitude for Each Sample

samples["lon"] = samples["geometry"].map(get_lon_from_one_column)





In [14]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.models import Model

# Load the pre-trained VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False)

In [1]:

feature_layer = 'block1_pool'
feature_extractor = Model(inputs=vgg16_model.input, outputs=vgg16_model.get_layer(feature_layer).output)

# Features and Labels
features = []
labels = []

# Loop through all images
def load_data(filename, directory):
    img_path = directory + "/" + filename + ".png"

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract features using the VGG16 model
    vgg16_features = feature_extractor.predict(img_array)

    # Flatten the VGG16 features to a 1D array
    flattened_vgg16_features = vgg16_features.flatten()

    # Combine VGG16 features with existing features (central pixel values)

    features.append(flattened_vgg16_features)

    # Assign labels based on the path
    if "/Ass/" in img_path:
        labels.append(1)
    elif "/Non-Ass/" in img_path:
        labels.append(0)

# Convert lists to NumPy arrays
X = np.array(features)
y = np.array(labels)


NameError: name 'Model' is not defined

In [2]:
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    samples[single_file] = samples["filenames"].map(
        lambda name, directory=os.path.join(base_directory, single_file): load_data(name, directory)
    )


NameError: name 'os' is not defined

In [ ]:
num_layers = len(vgg16_model.layers)


In [ ]:
from tensorflow.keras.applications import VGG16

# Load pre-trained VGG16 model without the top (fully connected) layers

# Iterate through the layers and print information
for i, layer in enumerate(vgg16_model.layers):
    print(f'Layer {i + 1}: {layer.name}, Type: {layer.__class__.__name__}, Output Shape: {layer.output_shape}')


Layer 1: input_1, Type: InputLayer, Output Shape: [(None, None, None, 3)]
Layer 2: block1_conv1, Type: Conv2D, Output Shape: (None, None, None, 64)
Layer 3: block1_conv2, Type: Conv2D, Output Shape: (None, None, None, 64)
Layer 4: block1_pool, Type: MaxPooling2D, Output Shape: (None, None, None, 64)
Layer 5: block2_conv1, Type: Conv2D, Output Shape: (None, None, None, 128)
Layer 6: block2_conv2, Type: Conv2D, Output Shape: (None, None, None, 128)
Layer 7: block2_pool, Type: MaxPooling2D, Output Shape: (None, None, None, 128)
Layer 8: block3_conv1, Type: Conv2D, Output Shape: (None, None, None, 256)
Layer 9: block3_conv2, Type: Conv2D, Output Shape: (None, None, None, 256)
Layer 10: block3_conv3, Type: Conv2D, Output Shape: (None, None, None, 256)
Layer 11: block3_pool, Type: MaxPooling2D, Output Shape: (None, None, None, 256)
Layer 12: block4_conv1, Type: Conv2D, Output Shape: (None, None, None, 512)
Layer 13: block4_conv2, Type: Conv2D, Output Shape: (None, None, None, 512)
Layer 14: 

SyntaxError: incomplete input (1192934036.py, line 2)